In [2]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm

#defines global variables of width and height.
width, height = 1000, 1000 
max_iter = 300
zoom_factor = 0.05  # Zoom in by 50% each click

# Initial zoom-out bounds
real_min, real_max = -25, 25 # fixed og bounds by the real axis
imag_min, imag_max = -25, 25 # fixed og bounds by the imaginary axis

# Fractal generation function
def compute_fractal(real_min, real_max, imag_min, imag_max):
    real = np.linspace(real_min, real_max, width) # real axis —— uses width as 
    imag = np.linspace(imag_min, imag_max, height) # imaginary axis
    real_grid, imag_grid = np.meshgrid(real, imag) # creates meshgrid out of real and imaginary axis—— the complex plane!
    c = real_grid + 1j * imag_grid #
    z = np.zeros_like(c)
    divergence_iter = np.zeros(c.shape, dtype=int)
    mask = np.ones(c.shape, dtype=bool)

    for i in range(max_iter):
        with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
            z[mask] = np.cos(z[mask]) + np.log(c[mask])
            mask_new = np.abs(z) <= 10
            divergence_iter[mask & ~mask_new] = i
            mask &= mask_new

    divergence_iter[mask] = max_iter
    return divergence_iter

# Plotting function
def plot_fractal(ax, divergence_iter, real_min, real_max, imag_min, imag_max):
    ax.clear()
    ax.imshow(
        divergence_iter,
        cmap=cm.cubehelix,
        extent=(real_min, real_max, imag_min, imag_max),
        origin='lower'
    )
    ax.set_title("Click to Zoom into Fractal")
    ax.set_xlabel("Re")
    ax.set_ylabel("Im")
    plt.draw()

# Click event callback
def on_click(event):
    global real_min, real_max, imag_min, imag_max

    if event.inaxes:
        # Get clicked coordinates
        x_center, y_center = event.xdata, event.ydata

        # Compute new zoomed bounds
        real_range = (real_max - real_min) * zoom_factor
        imag_range = (imag_max - imag_min) * zoom_factor
        real_min = x_center - real_range / 2
        real_max = x_center + real_range / 2
        imag_min = y_center - imag_range / 2
        imag_max = y_center + imag_range / 2

        # Recompute and redraw
        divergence_iter = compute_fractal(real_min, real_max, imag_min, imag_max)
        plot_fractal(ax, divergence_iter, real_min, real_max, imag_min, imag_max)

# Initial render
fig, ax = plt.subplots(figsize=(10, 10))
divergence_iter = compute_fractal(real_min, real_max, imag_min, imag_max)
plot_fractal(ax, divergence_iter, real_min, real_max, imag_min, imag_max)

# Connect click event
fig.canvas.mpl_connect('button_press_event', on_click)
plt.tight_layout()
plt.show()
